In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Fri Mar 26 20:47:16 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
ln: failed to create symbolic link '/mydrive/My Drive': File exists
'Colab Notebooks'		'Copy of yolov3_training_6000.weights'	 yolov3
'Copy of yolov3_10000.weights'	'My Drive'
'Copy of yolov3_7000.weights'	'Untitled document.gdoc'


**Cloning the darknet**  

In [ ]:
!git clone https://github.com/AlexeyAB/darknet

fatal: destination path 'darknet' already exists and is not an empty directory.


Compiling darknet-nvidia GPU

In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1127:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

**Configure yolov3**

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

We are using 5 classes <br>
1) With helmet <br>
2) Without helmet

So, for 5 class we have to change few configuration.<br>
Need to change -
*   Classes to 5
*   Filter to (n+5)*3 , n=classes so filter = 30
*  Max_baches to approx n*2000 , n->classs






In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 10000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=5@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=5@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=5@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=30@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=30@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=30@' cfg/yolov3_training.cfg

**Creating .name & .data files**

In [ ]:
!echo -e 'with_helmet\nwithout_helmet\nperson\nbike\nlicense_plate' > data/obj.names
!echo -e 'classes= 5\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data

Saving yolov3_training.cfg and obj.names to drive for later use

In [ ]:
!cp cfg/yolov3_training.cfg /mydrive/yolov3/yolov3_testing.cfg
!cp data/obj.names /mydrive/yolov3/classes.txt

Trained weight of convolutional layer 

In [ ]:
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2021-03-26 20:51:27--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  48.0MB/s    in 3.4s    

2021-03-26 20:51:31 (46.2 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**Unziping image folder**

In [ ]:
!mkdir data/obj
!unzip /mydrive/yolov3/finalDataset.zip -d data/obj

Archive:  /mydrive/yolov3/finalDataset.zip
   creating: data/obj/finalDataset/
  inflating: data/obj/finalDataset/BikesHelmets0.png  
  inflating: data/obj/finalDataset/BikesHelmets0.txt  
  inflating: data/obj/finalDataset/BikesHelmets1.png  
  inflating: data/obj/finalDataset/BikesHelmets1.txt  
  inflating: data/obj/finalDataset/BikesHelmets10.png  
  inflating: data/obj/finalDataset/BikesHelmets10.txt  
  inflating: data/obj/finalDataset/BikesHelmets100.png  
  inflating: data/obj/finalDataset/BikesHelmets100.txt  
  inflating: data/obj/finalDataset/BikesHelmets101.png  
  inflating: data/obj/finalDataset/BikesHelmets101.txt  
  inflating: data/obj/finalDataset/BikesHelmets102.png  
  inflating: data/obj/finalDataset/BikesHelmets102.txt  
  inflating: data/obj/finalDataset/BikesHelmets103.png  
  inflating: data/obj/finalDataset/BikesHelmets103.txt  
  inflating: data/obj/finalDataset/BikesHelmets104.png  
  inflating: data/obj/finalDataset/BikesHelmets104.txt  
  inflating: data/o

Generating file (train.txt) containing all images name 

In [ ]:
import glob
images_list = glob.glob("data/obj/finalDataset/*.png")
with open("data/train.txt", "w") as f:
    f.write("\n".join(images_list))   
#print(images_list)     

Training Model...

In [ ]:
# !./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show
!./darknet detector train data/obj.data cfg/yolov3_training.cfg /mydrive/yolov3/yolov3_training_last.weights -dont_show

Streaming output truncated to the last 5000 lines.
 total_bbox = 96813, rewritten_bbox = 0.100193 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.838186), count: 12, class_loss = 0.844181, iou_loss = 0.562793, total_loss = 1.406973 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.807407), count: 11, class_loss = 1.289235, iou_loss = 0.505699, total_loss = 1.794933 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.789049), count: 4, class_loss = 0.009543, iou_loss = 0.089386, total_loss = 0.098929 
 total_bbox = 96840, rewritten_bbox = 0.100165 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.851493), count: 13, class_loss = 0.503289, iou_loss = 0.326998, total_loss = 0.830286 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.828981), count: 3, class_loss = 0.016190, iou_loss = 0.065025, total_loss = 0.081215 
v3 (m